In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Recognizing news

In [0]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np
from tensorflow import keras

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [60]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. 

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [0]:
OUTPUT_DIR = '/content/drive/My Drive/nlp_test/bert_checkpoints'
if os.path.exists(OUTPUT_DIR):
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)

#Data

First, let's download the dataset from gdrive which was preprocessed 

In [0]:
train_sentences = pd.read_csv('/content/drive/My Drive/nlp_test/data/train_sentences.csv')

In [64]:
train_sentences.head()

,sentence,labels
0,В доме зарегистрировано 80 участников долевого...,0
1,Срыв сроков строительства зоопарка оценили в 1...,1
2,На сегодняшний день готовность составляет лишь...,0
3,А вот кто понесёт ответственность за срыв срок...,1
4,ЗОЛОТОЙ КВАДРАТНЫЙ МЕТР – Первый замначальника...,0


In [65]:
train_sentences.labels.value_counts(True)

0    0.802876
1    0.197124
Name: labels, dtype: float64

In [66]:
train_sentences[train_sentences.labels == 1].shape,train_sentences[train_sentences.labels == 0].shape

((329, 2), (1340, 2))

We can see Imbalanced Classification where the number of class 1 objects is less than class 0. Let's try to propagate a class with less examples.

In [67]:
train_sentences[train_sentences.labels == 1].sentence.repeat(3).shape

(987,)

In [68]:
train_sentences_new = pd.DataFrame()
oversample_sent = train_sentences[train_sentences.labels == 1].sentence.repeat(3)
train_sentences_new['sentence'] = oversample_sent
train_sentences_new['labels'] = [1 for i in range(len(oversample_sent))]

train_sentences = pd.concat([train_sentences, train_sentences_new]).reset_index(drop=True).sample(frac=1).reset_index(drop=True)
train_sentences.labels.value_counts(True)

0    0.504518
1    0.495482
Name: labels, dtype: float64

In [0]:
train, val = train_test_split(train_sentences, test_size=0.1)

In [70]:
train.head()

,sentence,labels
1066,Строительство профильного медцентра отложено н...,1
1250,"Наблюдательный совет ассоциации ""НП Совет рынк...",1
333,Оба дома будут сданы в 2013 году.,0
2649,"Пока строили дома, они подросли до головной бо...",0
268,«Стройград»: грандиозная ульяновская стройка (...,1


In [71]:
train.shape

(2390, 2)

For us, our input data is the 'sentence' column and our label (0, 1 for new with desired event and without, respecitvely)

In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'labels'
label_list = [0, 1]

#Data Preprocessing
Need to transform data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [74]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


We also created a tokenizer, which breaks words into word pieces:

In [75]:
tokenizer.tokenize("Это просто пример для меня")

['Это', 'просто', 'пример', 'для', 'меня']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [76]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 2390


INFO:tensorflow:Writing example 0 of 2390


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


#Creating a model

First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to task (classify whether new contains events in which talks about decommissioning of an object and other). This strategy of using a mostly trained model is called fine-tuning.

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params): 
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 20.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [88]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/nlp_test/bert_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8de98af550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/nlp_test/bert_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8de98af550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [91]:
!nvidia-smi

Wed Feb 26 07:59:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    25W /  75W |   7373MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Now let's use our test data to see how well our model did:

In [0]:
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [93]:
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 2000: auc = 0.945946, eval_accuracy = 0.9398496, f1_score = 0.9365079, false_negatives = 0.0, false_positives = 16.0, global_step = 2000, loss = 0.52718955, precision = 0.880597, recall = 1.0, true_negatives = 132.0, true_positives = 118.0


INFO:tensorflow:Saving dict for global step 2000: auc = 0.945946, eval_accuracy = 0.9398496, f1_score = 0.9365079, false_negatives = 0.0, false_positives = 16.0, global_step = 2000, loss = 0.52718955, precision = 0.880597, recall = 1.0, true_negatives = 132.0, true_positives = 118.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /content/drive/My Drive/nlp_test/bert_checkpoints/model.ckpt-2000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /content/drive/My Drive/nlp_test/bert_checkpoints/model.ckpt-2000


{'auc': 0.945946,
 'eval_accuracy': 0.9398496,
 'f1_score': 0.9365079,
 'false_negatives': 0.0,
 'false_positives': 16.0,
 'global_step': 2000,
 'loss': 0.52718955,
 'precision': 0.880597,
 'recall': 1.0,
 'true_negatives': 132.0,
 'true_positives': 118.0}

Now let's write code to make predictions on news sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

# Predictions

In [95]:
test_all_sent = pd.read_csv('/content/drive/My Drive/nlp_test/data/test_all_sentences.csv')
test_all_sent.head()

,title,sentence
0,Уралкуз” изготовил рекордное количество осей,"ПАО ""Уральская кузница"" (входит в Группу ""Мече..."
1,Уралкуз” изготовил рекордное количество осей,"""Уралкуз"" в октябре 2019 года поставил абсолют..."
2,Уралкуз” изготовил рекордное количество осей,За 10 месяцев 2019 года отгружено более 12 тыс...
3,Уралкуз” изготовил рекордное количество осей,"""Мы побили рекорд 2013 года."
4,Уралкуз” изготовил рекордное количество осей,"Тогда за весь год мы отгрузили почти 11,2 тыс...."


In [148]:
test_all_sent.shape

(166962, 2)

In [96]:
test_titles = pd.read_csv('/content/drive/My Drive/nlp_test/data/test_titles.csv')
test_titles.head()

,title
0,Уралкуз” изготовил рекордное количество осей
1,Отключение Ирана от интернета выявило сильные ...
2,Во Владивостоке количество наружной рекламы до...
3,"ФУТБОЛ-Месси получил рекордный шестой ""Золотой..."
4,«ОРУ под напряжением Луны»: фото амурчанина по...


In [0]:
test_titles.title.apply(lambda x:  1 if type(x) == float else 0).sum()

206

In [0]:
test_titles.title = test_titles.title.astype(str)

In [0]:
test_all_sent.title = test_all_sent.title.astype(str)
test_all_sent.sentence = test_all_sent.sentence.astype(str)

In [99]:
predictions = getPrediction(test_all_sent.sentence.values)

INFO:tensorflow:Writing example 0 of 166962


INFO:tensorflow:Writing example 0 of 166962


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/nlp_test/bert_checkpoints/model.ckpt-2000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [126]:
predictions_titles = getPrediction(test_titles.title.values)

INFO:tensorflow:Writing example 0 of 10782


INFO:tensorflow:Writing example 0 of 10782


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/nlp_test/bert_checkpoints/model.ckpt-2000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [100]:
pd.Series.value_counts([p[2] for p in predictions])

Negative    166262
Positive       700
dtype: int64

In [0]:
test_all_sent_pred = test_all_sent.copy()
test_title_pred = test_titles.copy()

In [0]:
test_title_pred['prediction'] = [p[2] for p in predictions_titles]

In [0]:
test_all_sent_pred['prediction'] = [p[2] for p in predictions]


# Ranking news by importance

In [136]:
# news should contain at least 4 relevant(positive) sentenses 
threshold = 3 
# prediction by all sentences in news
t = test_all_sent_pred[test_all_sent_pred.prediction == 'Positive'].groupby('title')['prediction'].size().sort_values(ascending=False)
t[t>threshold]

title
За нацпроекты ответишь                                                                                        10
ГЕРМАНИЯ: НОВЫЙ АЭРОПОРТ БЕРЛИНА ОБЕЩАЮТ ОТКРЫТЬ В 2020 ГОДУ                                                   8
Один долгострой и несколько торговых центров сдали во Владивостоке в 2019 году                                 8
В администрации Орла рассказали об итогах реализации нацпроекта «БКАД» в 2019 году                             6
ГРЭС 2 смогла проработать 4 дня после запуска                                                                  6
Выплату компенсаций по советским вкладам перенесли еще на год                                                  6
Когда дольщики долгостроя "Восточка" в Алматы отметят новоселье                                                6
Паромы пойдут, а проблемы останутся...                                                                         6
Безальтернативные турбины                                                                 

In [137]:
# prediction by all sentences in news
# due to fact that only one sentence is used for prediction, the quality is worse, and it is also difficult to make rankings without additional model wrapper
title_pred = test_title_pred[test_all_sent_pred.prediction == 'Positive'].title.values
title_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


array(['«Сафмар» тянет газ на себя // Группа добивается снижения цены для своего проекта в Белоруссии',
       'В Кричевском районе при пожаре погиб мужчина',
       'Расхитителей ливневок в Хабаровске поймали на превышении скорости',
       'За «Черную пятницу» россияне потратили в интернет-магазинах 22 млрд рублей',
       'Япония не устояла',
       'Прокуратура: МП «Нижегородское метро» нарушает права инвалидов',
       'Жителям Новосибирска пообещали оставить старую цену проезда по «Социальной карте»',
       'В Москву прилетел премьер-министр Монголии',
       'Пассажир самолета Москва - Горно-Алтайск закурил на борту IQOS',
       'Выгнанную с чаепития новосибирскую школьницу перевели в другую школу',
       'Аргентинский футболист Лионель Месси получил «Золотой мяч» в шестой раз',
       'Новый глава дипломатии ЕС заявил, что пока нет причин снимать санкции против России',
       'Операторы связи просят поправить «закон о\xa0суверенном Рунете» и\xa0защитить данные абонентов',
 

In [141]:
# only for news with "Positive" titles
# prediction looks not well
test_all_sent_pred[test_all_sent_pred.title.isin(title_pred)&(test_all_sent_pred.prediction == 'Positive')].title.value_counts()

«АртСтрой» могут оштрафовать за опоздание с ремонтом тротуаров    3
Билеты оценили в парламенте                                       3
nan                                                               1
Name: title, dtype: int64